# Multi-scale (MS) CNN autoencoder
## Author: Taichi Nakamura (Keio university, Japan)
Taichi Nakamura provides no guarantees for this code.  Use as-is and for academic research use only; no commercial use allowed without permission. For citations, please use the reference below:  
Ref: Taichi Nakamura, Kai Fukami, and Koji Fukagata, "Convolutional neural network and long short-term memory based reduced order surrogate for minimal turbulent channel flow," Physics of Fluids, 2021.  
The code is written for educational clarity and not for speed.  
-- version 1: Jan 27, 2021

For making MS CNN-AE, the user has to install 'keras', 'numpy', 'pandas' and 'sklearn'.

In [1]:
from keras.layers import Input, Add, Conv3D, MaxPooling3D, UpSampling3D, Reshape
from keras.models import Model
from keras import backend as K
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from keras.callbacks import ModelCheckpoint,EarlyStopping

Using TensorFlow backend.


## Load data

In [ ]:
gridSetting = (34,66,34)

datasetSerial = np.arange(100,1000100,100)
datasetPrefix = './data_box/UVWP'

u_all = np.zeros((len(datasetSerial),32,32,32))
v_all = np.zeros((len(datasetSerial),32,32,32))
w_all = np.zeros((len(datasetSerial),32,32,32))

for i in tqdm(range(len(datasetSerial))):
    name = datasetPrefix + '{0:07d}'.format(datasetSerial[i])
    df = pd.read_csv(name, header=None, delim_whitespace=True)
    dataset = df.values
    BOX = dataset[:,:]

    u_org = BOX[:,0].reshape(gridSetting,order='F')
    v_org = BOX[:,1].reshape(gridSetting,order='F')
    w_org = BOX[:,2].reshape(gridSetting,order='F')

    u_all[i,:,:,:] = u_org[1:33,1:33,1:33]
    v_all[i,:,:,:] = v_org[1:33,1:33,1:33]
    w_all[i,:,:,:] = w_org[1:33,1:33,1:33]
    
u_mean = u_all.mean(axis=3).mean(axis=0).mean(axis=0)
v_mean = v_all.mean(axis=3).mean(axis=0).mean(axis=0)
w_mean = w_all.mean(axis=3).mean(axis=0).mean(axis=0)

u_flc = np.zeros(u_all.shape)
v_flc = np.zeros(v_all.shape)
w_flc = np.zeros(w_all.shape)

for j in tqdm(range(32)):
    u_flc[:,:,j,:] = u_all[:,:,j,:] - u_mean[j]
    v_flc[:,:,j,:] = v_all[:,:,j,:] - v_mean[j]
    w_flc[:,:,j,:] = w_all[:,:,j,:] - w_mean[j]

data_raw = np.zeros((len(datasetSerial),32,32,32,3)) #Training data
data_raw[:,:,:,:,0] = u_flc
data_raw[:,:,:,:,1] = v_flc
data_raw[:,:,:,:,2] = w_flc

## Construct model

In [ ]:
input_img = Input(shape=(32,32,32,3))

#Multi-scale model (Du et al., 2018)
x1 = Conv3D(16, (3,3,3),activation='relu', padding='same')(input_img)
x1 = Conv3D(8, (3,3,3),activation='relu', padding='same')(x1)
x1 = MaxPooling3D((2,2,2),padding='same')(x1)
x1 = Conv3D(16, (3,3,3),activation='relu', padding='same')(x1)
x1 = Conv3D(8, (3,3,3),activation='relu', padding='same')(x1)
x1 = MaxPooling3D((2,2,2),padding='same')(x1)
x1 = Conv3D(3, (3,3,3),activation='tanh', padding='same')(x1)

x2 = Conv3D(16, (5,5,5),activation='relu', padding='same')(input_img)
x2 = Conv3D(8, (5,5,5),activation='relu', padding='same')(x2)
x2 = MaxPooling3D((2,2,2),padding='same')(x2)
x2 = Conv3D(16, (5,5,5),activation='relu', padding='same')(x2)
x2 = Conv3D(8, (5,5,5),activation='relu', padding='same')(x2)
x2 = MaxPooling3D((2,2,2),padding='same')(x2)
x2 = Conv3D(3, (5,5,5),activation='tanh', padding='same')(x2)

x3 = Conv3D(16, (7,7,7),activation='relu', padding='same')(input_img)
x3 = Conv3D(8, (7,7,7),activation='relu', padding='same')(x3)
x3 = MaxPooling3D((2,2,2),padding='same')(x3)
x3 = Conv3D(16, (7,7,7),activation='relu', padding='same')(x3)
x3 = Conv3D(8, (7,7,7),activation='relu', padding='same')(x3)
x3 = MaxPooling3D((2,2,2),padding='same')(x3)
x3 = Conv3D(3, (7,7,7),activation='tanh', padding='same')(x3)

x = Add()([x1,x2,x3])
x_lnt = Reshape((1536,))(x) # For LSTM training

x = Reshape((8,8,8,3))(x_lnt)

x1 = Conv3D(3, (3,3,3),activation='relu', padding='same')(x)
x1 = UpSampling3D((2,2,2))(x1)
x1 = Conv3D(8, (3,3,3),activation='relu', padding='same')(x1)
x1 = Conv3D(16, (3,3,3),activation='relu', padding='same')(x1)
x1 = UpSampling3D((2,2,2))(x1)
x1 = Conv3D(8, (3,3,3),activation='relu', padding='same')(x1)
x1 = Conv3D(16, (3,3,3),activation='relu', padding='same')(x1)

x2 = Conv3D(3, (5,5,5),activation='relu', padding='same')(x)
x2 = UpSampling3D((2,2,2))(x2)
x2 = Conv3D(8, (5,5,5),activation='relu', padding='same')(x2)
x2 = Conv3D(16, (5,5,5),activation='relu', padding='same')(x2)
x2 = UpSampling3D((2,2,2))(x2)
x2 = Conv3D(8, (5,5,5),activation='relu', padding='same')(x2)
x2 = Conv3D(16, (5,5,5),activation='relu', padding='same')(x2)

x3 = Conv3D(3, (7,7,7),activation='relu', padding='same')(x)
x3 = UpSampling3D((2,2,2))(x3)
x3 = Conv3D(8, (7,7,7),activation='relu', padding='same')(x3)
x3 = Conv3D(16, (7,7,7),activation='relu', padding='same')(x3)
x3 = UpSampling3D((2,2,2))(x3)
x3 = Conv3D(8, (7,7,7),activation='relu', padding='same')(x3)
x3 = Conv3D(16, (7,7,7),activation='relu', padding='same')(x3)

x = Add()([x1,x2,x3])
x_final = Conv3D(3, (3,3,3),padding='same')(x)
autoencoder = Model(input_img, x_final)
autoencoder.compile(optimizer='adam', loss='mse')

## Learn parameters

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(data_raw, data_raw, test_size=0.3, random_state=None)
model_cb=ModelCheckpoint('./Model.hdf5', monitor='val_loss',save_best_only=True,verbose=1)
early_cb=EarlyStopping(monitor='val_loss', patience=50,verbose=1)
cb = [model_cb, early_cb]
history = dscms.fit(X_train,y_train,nb_epoch=5000,batch_size=128,verbose=1,callbacks=cb,shuffle=True,validation_data=[X_test, y_test])
df_results = pd.DataFrame(history.history)
df_results['epoch'] = history.epoch
df_results.to_csv(path_or_buf='./Model.csv',index=False)